In [1]:
import sys
sys.path.extend(['..'])

In [2]:
import torch
from distributions import log_prob_banana as log_p
from mvn import MVNDiag as MVN
from pyro.infer.mcmc.nuts import NUTS
from pyro.infer.mcmc import MCMC
import matplotlib.pyplot as plt

%matplotlib inline

In [3]:
q, lam_kl = MVN(d=2), 2.
def log_psi(th):
    q.set_theta(th)
    kl_qp = -q.entropy() - q.monte_carlo_ev(log_p, 50)
    return 0.5*q.log_det_fisher() - lam_kl * kl_qp

def potential(para):
    return -log_psi(para['theta'])

In [ ]:
kernel = NUTS(potential_fn=potential, adapt_step_size=True, adapt_mass_matrix=True, full_mass=False)
sampler = MCMC(kernel, num_samples=500, warmup_steps=300, initial_params={'theta': torch.zeros(q.n_params)})
sampler.run(None)

Warmup:  14%|█████▋                                    | 109/800 [04:13, 25.09s/it, step size=7.44e-04, acc. prob=0.772]                                                          

In [ ]:
plt.plot(sampler.get_samples()['theta'])
plt.legend(['$\mu_x$','$\mu_y$','$\log\sigma_x$','$\log\sigma_y$'])
plt.title('results using Pyro NUTS')
plt.xlabel('sample #')